# Scape info on cities from Wikipedia

Use english city names to obtain info from Wikipedia-pages:
* Country
* Popilation
* Coordinates (latitude, longitude)

Store in two seperate dataframes to export to MySQL

`cities_df`
* city_name (str)
* country (str)
* country_code (str)
* latitude (float)
* longitude (float)

`pop_df`
* city_name (str)
* population (int)
* measurement_date (string)
* retrieval_date (date)

Please ensure you have set up the corresponding tables with `sql/create_database_data_pipeline_example.sql`.

Save you MySQL password in `python/key.env` as `MYSQL_KEY` (or provide you password by other means) and open your MySQL workbench.

# Import libraries

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy
from dotenv import load_dotenv
import os
import re
from datetime import datetime

## City info

In [5]:
def getCityInfoWiki(cities):
    '''
    Compile info on Cities from Wikipedia:
    Country, Latitude, Longitude, Pupulation and date when population was assessed
    INPUT: list of city names in English
    OUTPUT:pandas data frame with columns:
        city (str)
        country (str)
        latitude (float)
        longitude (float)

    To-do:
    - add error if wiki page not found
    '''
    country = []
    country_code = []
    latitude = []
    longitude = []

    def convert_coordinates_degree2dec(coord):
        """Convert coordinates from DMS (e.g., '13°24′18″E', '13°24′E', or '13°E') to decimal degrees."""
        import re
        import math

        # Regex that allows optional minutes (′) and seconds (″)
        match = re.match(
            r"^\s*(\d+)°(?:\s*(\d+)′)?(?:\s*(\d+)″)?\s*([NSEW])\s*$",
            coord
        )

        if match:
            degrees, minutes, seconds, direction = match.groups()

            # Default missing parts to 0
            minutes = float(minutes) if minutes else 0.0
            seconds = float(seconds) if seconds else 0.0

            # Convert to decimal degrees
            decimal = float(degrees) + minutes/60 + seconds/3600

            # West/South are negative
            if direction in ['W', 'S']:
                decimal = -decimal
        else:
            decimal = float('nan')  # Return NaN for invalid format
            print(f"Invalid coordinate format: {coord}")

        return decimal


    for city in cities:
        url = "https://en.wikipedia.org/wiki/" + city.replace(" ", "_")
        headers = {'User-Agent': 'Chrome/134.0.0.0'}
        response = requests.get(url, headers=headers)
        soup_city = BeautifulSoup(response.content, 'html.parser')

        # Country
        country_name = soup_city.find("th", string="Country").find_next("td").get_text()
        country.append(country_name)
        country_code.append(country_name.upper()[0:3])

        # Location
        latitude.append(convert_coordinates_degree2dec(soup_city.find(class_='latitude').get_text()))
        longitude.append(convert_coordinates_degree2dec(soup_city.find(class_='longitude').get_text()))
    
    cities_df = pd.DataFrame({
        'city_name': cities, 
        'country':country, 
        'country_code':country_code, 
        'latitude':latitude, 
        'longitude': longitude
        })
    return cities_df

In [6]:
load_dotenv('keys.env')  # Load .env into environment
cities = os.getenv("CITIES").split(",")
cities_df = getCityInfoWiki(cities)
cities_df

,city_name,country,country_code,latitude,longitude
0,Berlin,Germany,GER,52.5200,13.405
1,Hamburg,Germany,GER,53.5500,10.000
2,Munich,Germany,GER,48.1375,11.575


## Population Info

In [ ]:
def getCityPopWiki(cities):
    '''
    Compile info on Cities from Wikipedia:
    Country, Latitude, Longitude, Pupulation and date when population was assessed
    INPUT: list of city names in English
    OUTPUT:pandas data frame with columns:
        city (str)
        population (int)
        measurement_date (string)
        retrieval_date (date)

    To-do:
    - add error if wiki page not found
    '''

    population = []
    measurement_date = []
    retrieval_date = []

    for city in cities:
        url = "https://en.wikipedia.org/wiki/" + city.replace(" ", "_")
        headers = {'User-Agent': 'Chrome/134.0.0.0'}
        response = requests.get(url, headers=headers)
        soup_city = BeautifulSoup(response.content, 'html.parser')

        # Population
        pop = soup_city.find(string="Population").find_next('td').get_text()
        population.append(int(pop.replace(",", "")))
        date = soup_city.find(string="Population").find_next('div').get_text()
        measurement_date.append(re.search(r'\((.*?)\)', date).group(1))
        retrieval_date.append(datetime.today().strftime('%Y-%m-%d'))

    cities_df = pd.DataFrame(
        {'city_name': cities, 
         'population':population, 
         'measurement_date':measurement_date, 
         'retrieval_date':retrieval_date})
    
    return cities_df

In [ ]:
pop_df = getCityPopWiki(cities)
pop_df 

## Send data to SQL

Please ensure you have set up the corresponding tables with `sql/create_database_data_pipeline_example.sql`.

Save you MySQL password in `python/key.env` as `MYSQL_KEY` (or provide you password by other means) and open your MySQL workbench.

### Provide info to connect to MySQL

In [ ]:
load_dotenv('keys.env')  # Load .env into environment
password = os.getenv("MYSQL_KEY")
schema = "data_pipeline_example"
host = os.getenv("HOST")
user = os.getenv("USER")
port = os.getenv("PORT")

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### Send city data

In [ ]:
cities_df.to_sql('cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

### Send population data

Retrieve city data first and merge with population data frage to get the city_id. Then send population data to MySQL.

In [ ]:
cities_from_sql = pd.read_sql("cities", con=connection_string)

pop_merged_df = pop_df.merge(cities_from_sql[['city_id','city_name']],
                      on = "city_name",
                      how="left")
pop_merged_df = pop_merged_df.drop(columns=["city_name"])

pop_merged_df.to_sql('population',
                if_exists='append',
                con=connection_string,
                index=False)